In [3]:
#Create Table
        


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local")\
       .appName("test")\
           .getOrCreate()

In [ ]:
#set table and database name
tableName = "xyz"
databaseName = "abc"


In [ ]:
#Check if table exists
exists = spark.catalog.tableExists(f"{databaseName}.{tableName}")

print(exists)

In [ ]:
import datetime

#Get end date (previous month)
end_date_df = spark.sql(
  """ 
  SELECT DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1' MONTH) as end_date
  """)
end_date = end_date_df.first()["end_date"]
print(f"END DATE: {end_date}")


if exists:
    #Get last date of data loaded in table
    start_date_df = spark.sql(
    f""" 
    SELECT MAX(<date_field>) + INTERVAL '1' MONTH as start_date FROM {databaseName}.{tableName}
    """)
    
    start_date = start_date_df.first()["start_date"]
    print(f"START DATE: {start_date}")
else:
    #Default to the beginning of reuired fiscal year
    start_date = datetime.datetime(2022, 11, 1, 00, 00, 00)
    print(f"START DATE: {start_date}")
    
if start_date > end_date:
    dbutils.notebook.exit("Stopping the notebook execution because there is no need to update the table")
        

In [ ]:
#Loading the query results from source onto data frame
df = spark.sql(f"""
   SELECT * FROM cde.wuv;
   """
   )

df.show()

In [ ]:
#Check number of rows in the table before load
if exists:
    row_count_df = spark.sql(f"SELECT count(*) as row_count FROM {databaseName}.{tableName}")
    row_count = row_count_df.first()["row_count"]
    
    print(row_count)
else:
    print(df.count())

In [ ]:
#mount path location in data catalog
mount_path = f'/mnt/pqr/{databaseName}/{tableName}'

In [ ]:
#If table exists append the data to the table
if exists:
    df.write.format("delta").mode("append").save(mount_path)
    print('Data appended')
else:
    #Overwrite the directory with the dataframe data
    df.write.format("delta").mode("overwrite").save(mount_path)
     print('Data written')

In [ ]:
#If table does not exist create table
if not exists:
    spark.sql(f"""
    CREATE TABLE {databaseName}.{tableName}
    USING DELTA LOCATION '{mount_path}'
    """)
    
    spark.sql(f"""
    ALTER TABLE {databaseName}.{tableName}
    SET TBLPROPERTIES ('edl_sources' = 'klm')
    """)
    
result = spark.sql(f"""
     SELECT * FROM {databaseName}.{tableName}
     """)
result.show()

In [ ]:
#To delete data and drop table
Drop table <Table_Name>

dbutils.fs.rm(mount_path, recurse=True)